In [1]:
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPooling2D
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.
/Users/vishalbarad/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/vishalbarad/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/vishalbarad/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/vishalbarad/opt/ana

In [2]:
train_data = "/Users/vishalbarad/Desktop/Train"
validation_data = "/Users/vishalbarad/Desktop/Test"

In [3]:
#Dataaugmentation because we need more images
traindatagen = ImageDataGenerator(
    shear_range=0.3,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.4,
    zoom_range=0.3,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,)

validatedatagen = ImageDataGenerator(rescale=1./255)


In [4]:
num_classes = 2 # because we have 5 categories as angry,happy,netural,sad,surprise 
img_rows,img_cols=48,48
batch_size = 32

traindata_generator = traindatagen.flow_from_directory(
    train_data,
    class_mode="categorical",
    color_mode="grayscale",
    batch_size=batch_size,
    target_size=(img_rows,img_cols),
    shuffle=True)

validationdata_generator = validatedatagen.flow_from_directory(
    validation_data,
    class_mode="categorical",
    color_mode="grayscale",
    batch_size=batch_size,
    target_size=(img_rows,img_cols),
    shuffle=True)

Found 12220 images belonging to 2 classes.
Found 1505 images belonging to 2 classes.


In [5]:
model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))



Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))



In [7]:
# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [8]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

'''checkpoint = ModelCheckpoint('happyfacedetaction.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]'''

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])


In [9]:
nb_train_samples = 12220
nb_validation_samples = 1505
epochs=10


history=model.fit_generator(
                traindata_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                validation_data=validationdata_generator,
                validation_steps=nb_validation_samples//batch_size)


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
381/381 [==============================] - 12487s 33s/step - loss: 0.8126 - accuracy: 0.5446 - val_loss: 0.6654 - val_accuracy: 0.5559
Epoch 2/10
381/381 [==============================] - 488s 1s/step - loss: 0.6882 - accuracy: 0.5788 - val_loss: 0.7364 - val_accuracy: 0.5777
Epoch 3/10
381/381 [==============================] - 407s 1s/step - loss: 0.6707 - accuracy: 0.5950 - val_loss: 0.5787 - val_accuracy: 0.5852
Epoch 4/10
381/381 [==============================] - 449s 1s/step - loss: 0.6692 - accuracy: 0.5972 - val_loss: 0.6344 - val_accuracy: 0.5798
Epoch 5/10
381/381 [==============================] - 500s 1s/step - loss: 0.6583 - accuracy: 0.6099 - val_loss: 0.8372 - val_accuracy: 0.5900
Epoch 6/10
381/381 [==============================] - 636s 2s/step - loss: 0.6532 - accuracy: 0.6128 - val_loss: 0.6936 - val_accuracy: 0.5798
Epoch 7/10
381/381 [==============================] - 400s 1s/step - loss: 0.6391 - accurac

In [10]:
model.save("Happyface_detection_project.h5")